<a href="https://colab.research.google.com/github/robertmaxwilliams/talking-statues/blob/master/colab/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Make sure google drive is connected and data is accessible.

In [1]:
!nvidia-smi

Fri Mar  6 15:44:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
!pip install gpt_2_simple 

  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.1-cp36-none-any.whl size=23581 sha256=407d91f9f805b64eb7cb4a695fe18141a5e45a2ea7e822a05c0c554a6ac22e97
  Stored in directory: /root/.cache/pip/wheels/0c/f8/23/b53ce437504597edff76bf9c3b8de08ad716f74f6c6baaa91a
Successfully built gpt-2-simple


In [5]:
import gpt_2_simple as gpt2
import os

class bcolors:
    HEADER = "\033[95m"
    OKBLUE = "\033[94m"
    OKGREEN = "\033[92m"
    WARNING = "\033[93m"
    FAIL = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [6]:
model_name = "124M"
if not os.path.isdir(os.path.join("models", model_name)):
    print(f"Downloading {model_name} model...")
    gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 337Mit/s]                                                      
Fetching encoder.json:   0%|                                           | 0.00/1.04M [00:00<?, ?it/s]

Fetching encoder.json: 1.05Mit [00:00, 65.7Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 364Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:02, 203Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 216Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 126Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 124Mit/s]                                                       


In [0]:
!mkdir checkpoint && cp -r models/124M checkpoint/run1

In [0]:
data_path = "/content/drive/My Drive/GRIMM";
sess = gpt2.start_tf_sess()
# gpt2.load_gpt2(sess)
gpt2.finetune(sess,
              # dataset=data_path,
              dataset=data_path + "/CAT-SKIN.txt",
              model_name='124M',
              steps=100,
              restore_from='fresh',
              run_name='run1',
              print_every=1,
              sample_every=200,
              save_every=50,
              )

def generate_text(prefix):
    prefix = prefix[:-100]
    output = gpt2.generate(
        sess,
        prefix=prefix,
        include_prefix=False,
        return_as_list=True,
        length=100,
        batch_size=5,
        nsamples=5,
    )
    # print(f'\n\n{bcolors.OKGREEN}{prefix} {bcolors.ENDC}{output}\n')
    output = [x.lstrip(prefix) for x in output]
    return output


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  3.16it/s]


dataset has 2702 tokens
Training...
[1 | 10.28] loss=2.74 avg=2.74
[2 | 11.57] loss=2.80 avg=2.77
[3 | 12.83] loss=2.66 avg=2.73
[4 | 14.09] loss=2.58 avg=2.69
[5 | 15.35] loss=2.25 avg=2.60
[6 | 16.62] loss=1.92 avg=2.49
[7 | 17.89] loss=1.85 avg=2.39
[8 | 19.15] loss=1.82 avg=2.32
[9 | 20.44] loss=1.57 avg=2.23
[10 | 21.73] loss=1.48 avg=2.15
[11 | 23.01] loss=1.28 avg=2.07
[12 | 24.27] loss=1.13 avg=1.99
[13 | 25.54] loss=0.98 avg=1.91
[14 | 26.80] loss=0.85 avg=1.82
[15 | 28.06] loss=0.74 avg=1.75
[16 | 29.34] loss=0.61 avg=1.67
[17 | 30.59] loss=0.90 avg=1.62
[18 | 31.89] loss=0.41 avg=1.55
[19 | 33.16] loss=0.36 avg=1.48
[20 | 34.42] loss=0.49 avg=1.43
[21 | 35.69] loss=0.32 avg=1.37
[22 | 36.95] loss=0.25 avg=1.31
[23 | 38.24] loss=0.23 avg=1.26
[24 | 39.51] loss=0.12 avg=1.21
[25 | 40.78] loss=0.16 avg=1.16
[26 | 42.07] loss=0.22 avg=1.12
[27 | 43.35] loss=0.08 avg=1.07
[28 | 44.62] loss=0.19 avg=1.04
[29 | 45.90] loss=0.09 avg=1.00
[30 | 47.17] loss=0.05 avg=0.96
[31 | 48.45] 